In [1]:
import traceback
from pymongo import MongoClient
from elasticsearch import Elasticsearch

In [2]:
db = MongoClient('mongodb://127.0.0.1:27017')['Dogforum']

In [3]:
for type_name in db.collection_names():
    print(type_name)

dog_stories


# Method 1: combine key, value in a dict_list into tuple 

In [4]:
title_cursor = db[type_name].find({},{'_id':0,'title':1, 'content':1})

In [5]:
# title = [b.values() for b in title_cursor]
# tup_title = [tuple(tup) for tup in title]
tup_title = [tuple(tup) for tup in [a.values() for a in title_cursor]]

In [6]:
tup_title[1:3]

[("Lab thinks she's a lifeguard",
  " I love those moments, the one's you'll always remember once they're gone "),
 ("Lab thinks she's a lifeguard",
  " Sounds like she was retrieving something that was special to her and having a blast doing it. Also sounds like she's naturally inclined to water rescue. If she has the typical lab soft mouth grab it sounds like she would be great at water rescue.  \nAlso your vacation sounds amazing. I am jealous. ")]

# Method 2: concatenate values with a key in a dict_list

In [7]:
tcu_cursor = db[type_name].find({},{'_id':0,'title':1, 'content':1, 'url':1})

In [8]:
data = [b for b in tcu_cursor]

In [9]:
len(data)

1320

In [10]:
articles = []
for num in range(0, len(data)):
    #block = {}
    #block[data[num].get('title')] = data[num].get('content')+ '[ ' + data[num].get('url') + ' ]'
    #articles.append(tuple(block))
    block = (data[num].get('title'), data[num].get('content')+ '[ ' + data[num].get('url') + ' ]')
    articles.append(block)

In [11]:
type(articles)

list

In [12]:
len(articles)

1320

# Map-Reduce (key: title_name, value: contents)

In [13]:
import collections as c

In [14]:
agg_title = c.defaultdict(list)
for k, v in articles:
    agg_title[k].append(v)

In [15]:
type(agg_title)

collections.defaultdict

In [16]:
list(agg_title.items())[3]

('Is it bad to have a playful dog?',
 ['  My Puppy plays alot and likes playing outdoors. \r\nis it something to be worried about?   https://www.youtube.com/watch?v=BF6vjKZfXXQ [ http://www.dogforum.com/dog-stories/bad-have-playful-dog-290769/ ]',
  ' Why would you worry about that?  Dogs playing seem normal to me. [ http://www.dogforum.com/dog-stories/bad-have-playful-dog-290769/ ]',
  " A a happy chappie.....what's to worry about? [ http://www.dogforum.com/dog-stories/bad-have-playful-dog-290769/ ]",
  " Play is a good thing. The more happy play the better, as far as I'm concerned! \nI remember when I was 11 and adopted my Lab mix puppy (who has since passed). Her mother was a border collie and she was from a litter of TEN. When I came in, she was the only one that was awake, and I decided I wanted a playful dog so I chose her.  \nI don't regret that choice for a second. She was quite the character but overall was an amazing dog. Loving, playful, kind. Perfection. [ http://www.dogfor

In [17]:
len(agg_title.items())

184

In [18]:
# Title + "content_url"
len(list(agg_title.items())[3])

2

In [19]:
# Title is the key
list(agg_title.items())[3][0]

'Is it bad to have a playful dog?'

## New Challenge!!! ('Board' + 'Title') is key, ('Content' + 'url') is value

In [ ]:
""" Note: Combining two col of tuple into one key can't use DefaultDict to Map-Reduce
    all_cursor = db[type_name].find({},{'_id':0,'title':1, 'content':1, 'url':1, 'board':1})
    could be useless  """

### Setting group A

In [29]:
board_cursor = db[type_name].find({},{'_id':0,'title':1,'board':1})

In [21]:
data1 = [d for d in board_cursor]

In [22]:
# dictionary type can't use set to distinct duplication
data1[0]

{'board': 'dog-stories', 'title': "Lab thinks she's a lifeguard"}

In [30]:
tup_board = [tuple(tup) for tup in [e.values() for e in board_cursor]]

In [33]:
type(tup_board)

list

In [34]:
title_board = set(tup_board)

In [35]:
t_b_list = list(title_board)

In [36]:
len(t_b_list)

184

In [37]:
t_b_list[0]

('dog-stories', 'Funny Jack Russell youtube video')

In [38]:
sorted_A = sorted(t_b_list, key=lambda x:x[1])
sorted_A[:5]

[('dog-stories', '"I\'ll help myself to those treats! Thank you!"'),
 ('dog-stories', '*WARNING* Sad story about dog and his owner'),
 ('dog-stories', '2 years and 1 week, still missing my Shadow'),
 ('dog-stories', '24 year old Dachshund!'),
 ('dog-stories', "A Nerve-racking Journey - Max's Journal")]

### Group B

In [39]:
b = list(agg_title.items())

In [40]:
b[0][0]

"Lab thinks she's a lifeguard"

In [41]:
grp_B = []
for num in range(0, len(b)):
    tup = (b[num][0], b[num][1])
    grp_B.append(tup)

In [42]:
len(grp_B)

184

In [43]:
grp_B[3]

('Is it bad to have a playful dog?',
 ['  My Puppy plays alot and likes playing outdoors. \r\nis it something to be worried about?   https://www.youtube.com/watch?v=BF6vjKZfXXQ [ http://www.dogforum.com/dog-stories/bad-have-playful-dog-290769/ ]',
  ' Why would you worry about that?  Dogs playing seem normal to me. [ http://www.dogforum.com/dog-stories/bad-have-playful-dog-290769/ ]',
  " A a happy chappie.....what's to worry about? [ http://www.dogforum.com/dog-stories/bad-have-playful-dog-290769/ ]",
  " Play is a good thing. The more happy play the better, as far as I'm concerned! \nI remember when I was 11 and adopted my Lab mix puppy (who has since passed). Her mother was a border collie and she was from a litter of TEN. When I came in, she was the only one that was awake, and I decided I wanted a playful dog so I chose her.  \nI don't regret that choice for a second. She was quite the character but overall was an amazing dog. Loving, playful, kind. Perfection. [ http://www.dogfor

In [44]:
sorted_B = sorted(grp_B, key=lambda x:x[0])

In [45]:
for b in sorted_B[:5]:
    print(b[0])

"I'll help myself to those treats! Thank you!"
*WARNING* Sad story about dog and his owner
2 years and 1 week, still missing my Shadow
24 year old Dachshund!
A Nerve-racking Journey - Max's Journal


### Group by key(title) within two sorted lists to join

In [ ]:
# reference to join in dict
import operator
sorting_key = operator.itemgetter("id")
a = sorted(a, key=sorting_key)
b = sorted(b, key=sorting_key)
for i, j in zip(sorted(a, key=sorting_key), sorted(b, key=sorting_key)):
    i.update(j)

In [47]:
# Fail to join two lists in a form of tuple
for comb_res, j in zip(sorted_B, sorted_A):
    comb_res.append(j[0])

AttributeError: 'tuple' object has no attribute 'append'

## Should  tuple --> dict, too lousy! Try another way...

In [48]:
all_cursor = db[type_name].find({},{'_id':0,'title':1,'board':1,'content':1,'url':1})

In [49]:
data2 = [f for f in all_cursor]

In [50]:
# merge in form of (title + board), (content + url)  
articles2 = []
for num in range(0, len(data2)):
    board = (data2[num].get('board') + "分" + data2[num].get('title'), "分" + data2[num].get('content') + data2[num].get('url'))
    articles2.append(board)

In [51]:
articles2[0]

("dog-stories分Lab thinks she's a lifeguard",
 "分  So just a little story we think is neat, thought I'd share. \nMy wifes dog is a black lab, field type, came from good hunting lines, she says, and I believe her\nAnyway, couple of years ago we went camping on a lake in the mountains. Lake is filled by waterfalls, crystal clear water. Like tap water clear. Just an awesome vacation seeing fall leaf change, hiking, swimming, finding waterfalls by boat etc. \nI decided to go swimming, and found a nice spot with some sandy bank and a good sized boulder hanging over the lake. Walked to edge of the boulder, and jumped the 10 feet or so into the lake. Still hadnt popped back up when I heard a big kersplash near me. Popped up and theres mad-dog ( what I call the lab, real name maddie ) swimming up to me. Gets to me and starts like trying to scramble up on me like shes trying to drown me lol. So I grabbed her collar to push her away, almost instantly she turns and starts for the sandy part of the

In [52]:
# Separate Board + Title
str(articles2[0]).split('分')[0].replace('("',"")

'dog-stories'

In [53]:
str(articles2[0]).split('分')[1].replace('", "',"")

"Lab thinks she's a lifeguard"

In [54]:
str(articles2[0]).split('分')[2]

'  So just a little story we think is neat, thought I\'d share. \\nMy wifes dog is a black lab, field type, came from good hunting lines, she says, and I believe her\\nAnyway, couple of years ago we went camping on a lake in the mountains. Lake is filled by waterfalls, crystal clear water. Like tap water clear. Just an awesome vacation seeing fall leaf change, hiking, swimming, finding waterfalls by boat etc. \\nI decided to go swimming, and found a nice spot with some sandy bank and a good sized boulder hanging over the lake. Walked to edge of the boulder, and jumped the 10 feet or so into the lake. Still hadnt popped back up when I heard a big kersplash near me. Popped up and theres mad-dog ( what I call the lab, real name maddie ) swimming up to me. Gets to me and starts like trying to scramble up on me like shes trying to drown me lol. So I grabbed her collar to push her away, almost instantly she turns and starts for the sandy part of the bank. So I held on and she pulled me back.

### Stragegy is successful, there we go~

In [55]:
agg_by_title = c.defaultdict(list)
for k, v in articles2:
    agg_by_title[k].append(v)

In [56]:
len(articles2)

1320

In [57]:
len(agg_by_title)

184

In [58]:
type(agg_by_title)

collections.defaultdict

## resolve into 'board', 'title', 'agg_content + url'

In [59]:
aggs = list(agg_by_title.items())

In [61]:
len(aggs)

184

In [60]:
aggs[183][0].split('分')

['dog-stories', 'Leos story']

In [62]:
results = []
for num in range(0, len(aggs)):
    a = {}
    a['board'] = aggs[num][0].split('分')[0]
    a['title'] = aggs[num][0].split('分')[1]
    a['contents'] = " ".join(str(x) for x in aggs[num][1]).replace('分',"")
    results.append(a)

In [63]:
results[183]

{'board': 'dog-stories',
 'contents': "  This is a story of Leo, a miniature poodle that I sold to a breeder. She no longer wanted him after a year but didnt tell me and gave him to a farmer  I found out what happened to him some time later through someone lying about being a rescuer. I think stories like this need to be out there for others to know what happens to their beloved dogs if you arent careful and desex them.  A Dog I Sold Is Half Dead. - Dogz Online Forums http://www.dogforum.com/dog-stories/leos-story-27/  Leo's story always pulls at my heart strings. \r\nPeople can be so cruel and horrible, I couldn't imagine\r\none of my dogs ending up in the wrong hands like poor Leo!  \r\nWhat is the rest of Leo's story? Did he survive? Did you ever get\r\n him back? http://www.dogforum.com/dog-stories/leos-story-27/  Ah thats why the link is on there, you have to read it.  Take you hours though, lots of tears, relief, back to tears, wanting to shoot ppl, back to tears you name it. htt